In [ ]:
# załadowanie bibliotek

import pandas as pd
import numpy as np

In [ ]:
# wczytanie danych

dane = pd.ExcelFile('data_to_agg.xlsx').parse('aggregation_example')
dane

# Funkcje agregujące - Statystyki opisowe

### Miary położenia

**Miary tendencji centralnej:**

In [ ]:
print(dane['wiek'].mean())                                 # średnia
print(dane['wiek'].median())                               # mediana
print(dane['wiek'].mode())                                 # dominanta

**Inne miary położenia:**

In [ ]:
print(dane['wiek'].quantile(0.25))                          # kwantyl rzędu 0.25 (1. kwartyl)
print(dane['wiek'].quantile([0, 0.25, 0.5, 0.75, 1]))       # minimum, 1. kwartyl, 2.kwartyl, 3. kwartyl, maksimum

In [ ]:
print(dane['wiek'].min())                                   # minimum  - kwantyl rzędu 0
print(dane['wiek'].median())                                # mediana  - kwantyl rzędu 0.5
print(dane['wiek'].max())                                   # maksimum - kwantyl rzędu 1

### Miary rozproszenia

**Miary rozproszenia wokół miary tendencji centralnej:**

In [ ]:
print(dane['wiek'].var())                                   # wariancja nieobciążona
print(dane['wiek'].std())                                   # odchylenie standardowe

### Miary koncentracji

In [ ]:
print(dane['wiek'].kurt())                                  # kurtoza

### Miary asymetrii

In [ ]:
print(dane['wiek'].skew())                                  # skośność 

---

Podsumowanie powyższych miar daje funkcja **describe**:

In [ ]:
dane['wiek'].describe()                                     # najważniejsze statystyki opisowe

Można też samodzielnie wybrać kilka miar i skorzystać z funkcji **aggregate**:

In [ ]:
dane['wiek'].aggregate(['min', 'max', 'count'])             # wybrane statystyki opisowe

---


## Więcej statystyk opisowych

Więcej statystyk opisowych, np. współczynnik zmienności i rozstęp międzykwartylowy, można znaleźć w pakiecie scipy.stats: \
https://docs.scipy.org/doc/scipy/reference/stats.html#summary-statistics

---

## Statystyki opisowe na DataFrame

Powyższe funkcje statystyczne z pandas istnieją również dla pandas.dataframe:

In [ ]:
dane.mean(numeric_only = True)

In [ ]:
dane[['wiek', 'staż']].mean()                 

In [ ]:
dane.loc[dane['staż'] > 4, ['wiek', 'staż']].mean()    

---

# Funkcja groupby

Funkcja **groupby** grupuje wiersze na podstawie tych samych wartości w podanej kolumnie lub kolumnach. Po niej powinna nastąpić agregacja, która wtedy zostanie wykonana osobno dla każdej grupy:

In [ ]:
dane.groupby('zawód')['wiek'].mean()               

### Kilka zmiennych grupujących

In [ ]:
dane.groupby(['zawód', 'płeć'])['wiek'].mean()   

### Kilka zmiennych agregowanych

In [ ]:
dane.groupby('zawód')[['wiek', 'staż']].mean()              

### Kilka funkcji agregujących

In [ ]:
dane.groupby('zawód')['wiek'].aggregate(['min', 'max', 'count'])     

### Kilka zmiennych grupujących, kilka zmiennych agregujących, kilka funkcji agregujących

In [ ]:
dane.groupby(['zawód', 'płeć'])[['wiek', 'staż']].aggregate(['min', 'max', 'count'])#.reset_index() 

---

## Ćwiczenie

1. Znajdź waria stażu i wieku dla osób poszczególnych płci.

2. Znajdź liczność, medianę, wariancję oraz skośność lat stażu dla osób poszczególnych płci i zawodów.

---

# Funkcja apply

Funkcja **apply** wykonuje podaną funkcję w obrębie kolumny - pozwala na połączenie właściwości funkcji map z funkcją agregującą.

In [ ]:
dane[['wiek', 'staż']].map(lambda x: x + 1)                  # działania tylko na każdym elemencie osobno

In [ ]:
dane[['wiek', 'staż']].apply(lambda x: x + sum(x))           # działania na każdym elemencie z funkcją agregacyjną wywołaną w obrębie kolumny

Funkcję apply można też połączyć z funkcją groupby, aby funkcje agregacyjne zostały wywołane w obrębie otrzymanych przez nią podzbiorów.

In [ ]:
dane.groupby('zawód')['staż'].apply(lambda x: x + sum(x))

# Funkcja transform

Jeśli chcemy wynik funkcji apply z funkcją groupby dopisać jako kolumnę do danych potrzebujemy w zamian niej funkcji transform:

In [ ]:
dane['staż z sumą'] = dane.groupby('zawód')['staż'].transform(lambda x: x + sum(x))
dane['staż z sumą'] 

In [ ]:
dane

---

## Ćwiczenie

Wykonaj standaryzację wieku - od każdego wieku odejmij średnią i podziel otrzymany wynik przez odchylenie standardowe. 